In [1]:
import numpy as np
import h5py
import os
from tqdm.notebook import tqdm

from methods.base import BaseConfig
# from methods.pore_press_calc_functions import find_pore_pressure

In [87]:
# to pore press calc func 
from pde import PDEBase, PDE, ScalarField, VectorField, MemoryStorage, CartesianGrid
from pde.tools.numba import jit

In [72]:
# to pore press calc func 

def get_xi(perm, params):
    '''
    perm - np.aaray with permeability
    other is taken from params object

    Функция для подсчёта коэффициента диффузии. Подробности - Баренблатт стр.19
    Размерности:
    1 Мпа = 10**7 г/см*с2
    1 мД = 10**-11 см2
    1 сПз = 10**-2 г/см*с
    Итого размерность [xi] = [мД]*[МПа]/[сПз] =
                           =  10**-11 [см2] * 10**7 [г/см*с2] / 10**-2 [г/см*с] =
                           =  10**-2 [см2/с]
                           =  10**-6 [м2/с]
 
    Function for calculating the diffusion coefficient. Details - Barenblatt p. 19
    Dimensionality:
    1 MPa = 10^7 g/cm*s^2
    1 mD = 10^-11 cm^2
    1 cP = 10^-2 g/cm*s
    Therefore, the dimensionality of [xi] is:
    [xi] = [mD] * [MPa] / [cP] =
         = 10^-11 [cm^2] * 10^7 [g/cms^2] / 10^-2 [g/cms] =
         = 10^-2 [cm^2/s] =
         = 10^-6 [m^2/s]
    '''
    xi = 10**-6 * perm / (params.m0 * params.mu * (1/params.K_ro + 1/params.K_m)) # m^2/s
    return xi * params.time_scale # rescaling to time scale


def get_source_by_Q(params):
    '''
    dP/dt ~ K_m * ((dv/dt)/V) - pressure change when dv of fluid is pumped in volume V
    '''
    V = 2000 * 3.1415 * 0.1**2 / 4  # some "volume" of source (borehole). m^3
    dP = params.K_ro * params.Q / V # pressure change MPa/s
    return dP * params.time_scale # rescaling to time scale

def get_p0(perm, params):
    '''
    initial pressure (without hydristat)
    '''
    return np.ones_like(perm) * params.P0


def find_pore_pressure(perm, params):
    '''
    pore pressure calc
    '''
    # dx, dy, dz = params.dx_dy_dz
    x_length, y_length, z_length = params.side_lenght 
    grid = CartesianGrid([[0, x_length], [0, y_length], [0, z_length]], perm.shape)  # generate grid

    p0 = get_p0(perm, params) # initial pore pressure
    pore_pressure_field = ScalarField(grid, data=p0)
 
    source_field = ScalarField(grid, data=0) # spatial source field.
    source_field.insert(params.source_loc, 1)  #  one borehole == point source
    source_time = f"{get_source_by_Q(params)}"  # constant source

    # source_time = f"sin(2*pi*{10}*t)" # давление на скважине как то меняется или...
       
    # def source_from_list(t):
    #     ss = np.zeros(30)
    #     ss[5:15] = 1
    #     return ss[t]
    # source = source_from_list
    
    xi = get_xi(perm, params) # xi
    xi_field = ScalarField(grid, data=xi)

    bc_x = {'derivative': 0}
    bc_y = {'derivative': 0}
    bc_z = {'derivative': 0}
    bc = [bc_x, bc_y, bc_z]


    # divergence(xi_field*gradient(P)) = xi_field*laplace(P) + dot(gradient(xi_field), gradient(P)) <----- for more stable numerical scheme
    if params.eq_non_uniform:
        eq = PDE({'P': f"xi_field*laplace(P) + dot(gradient(xi_field), gradient(P)) + {source_time}*source_field"}, 
            bc=bc,
            consts={'source_field': source_field, 
                    'xi_field': xi_field}
            )
    else:
        eq = PDE({'P': f"xi_field*laplace(P) + {source_time}*source_field"}, 
            bc=bc,
            consts={'source_field': source_field, 
                    'xi_field': xi_field}
            )  

    storage = MemoryStorage()

    result = eq.solve(pore_pressure_field, t_range=params.t_range-1, dt=params.dt, solver="scipy", tracker=['progress', 'plot', storage.tracker(1)])

    return storage.data

In [36]:
def source_from_list(t):
    ss = np.zeros(30)
    ss[5:15] = 1
    return ss[t]

source_from_list(20)

0.0

In [ ]:
perms_path = 'downscaled_models_02_26_2024__16_06_58.h5'

with h5py.File(perms_path, 'r') as f:
    perm = f['perm'][2]

params = BaseConfig(Q=1, time_scale=86400, t_range=30, dt=0.001, eq_non_uniform=True)

pore_press = find_pore_pressure(perm, params)


In [126]:
class NU_Pore_Press_Diffusion(PDEBase):
    """Diffusion equation with spatial dependence"""
    def __init__(self, perm, params):
        super().__init__()
        self.perm = perm
        self.params = params

        # generate grid
        x_length, y_length, z_length = params.side_lenght 
        self.grid = CartesianGrid([[0, x_length], [0, y_length], [0, z_length]], self.perm.shape)  

        # boundary condition
        # bc_x = {'derivative': 0}
        # bc_y = {'derivative': 0}
        # bc_z = {'derivative': 0}
        # bc = [bc_x, bc_y, bc_z]
        # self.bc = bc  
        self.bc = 'derivative'

        # diffusion coefficient
        self.xi_field = self.get_xi_field()

        # rate of pressure change in source
        self.source_dpdt = self.get_source_dpdt()

        # source field
        self.source_field = self.get_source_field()

        # initial pore pressure
        self.pore_ini_field = self.get_pore_ini_field()

    def get_xi(self):
        xi = 10**-6 * self.perm / (self.params.m0 * self.params.mu * (1/self.params.K_ro + 1/self.params.K_m)) # m^2/s
        return xi * self.params.time_scale # rescaling to time scale
    
    def get_xi_field(self):
        xi = self.get_xi() # xi
        return ScalarField(self.grid, data=xi)

    def get_source_field(self):
        source_field = ScalarField(self.grid, data=0) # spatial source field.
        source_field.insert(params.source_loc, self.source_dpdt)  #  one borehole == point source
        return source_field
    
    def get_source_dpdt(self):
        '''
        dP/dt ~ K_m * ((dv/dt)/V) - pressure change when dv of fluid is pumped in volume V
        '''
        V = 2000 * 3.1415 * 0.1**2 / 4  # some "volume" of source (borehole). m^3
        dP = self.params.K_ro * self.params.Q / V # pressure change MPa/s
        return dP * self.params.time_scale # rescaling to time scale

    def get_pore_ini_field(self):
        p0 = np.ones_like(self.perm) * self.params.P0
        pore_pressure_field = ScalarField(self.grid, data=p0)
        return pore_pressure_field
    
    def _make_pde_rhs_numba(self, state):
        """ the numba-accelerated evolution equation """
        # make attributes locally available
        xi_field = self.xi_field.data
        source_field = self.source_field.data

        # create operators
        laplace = state.grid.make_operator("laplace", bc=self.bc)
        gradient = state.grid.make_operator("gradient", bc=self.bc)
        gradient_xi = state.grid.make_operator("gradient", bc="derivative")
        dot = VectorField(state.grid).make_dot_operator()

        @jit
        def pde_rhs(state_data, t=0):
            """ compiled helper function evaluating right hand side """
            lapace_P = laplace(state_data)
            grad_P = gradient(state_data)
            grad_xi = gradient_xi(xi_field)
            dP_dt = xi_field * lapace_P + dot(grad_xi, grad_P) + source_field
            return dP_dt

        return pde_rhs


    def evolution_rate(self, state, t=0):
        grad_xi = self.xi_field.gradient(bc="derivative")
        lapace_P = state.laplace(bc=self.bc)
        grad_P = state.gradient(bc=self.bc)
        dP_dt = self.xi_field * lapace_P + grad_xi @ grad_P + self.source_field
        return dP_dt
   

In [127]:
params = BaseConfig(Q=1, time_scale=86400, t_range=300, dt=0.0001, eq_non_uniform=True)

eq = NU_Pore_Press_Diffusion(perm, params)
p0 = eq.pore_ini_field

storage = MemoryStorage()

eq.solve(p0, t_range=params.t_range-1, dt=params.dt, solver='scipy', tracker=['progress', 'plot', storage.tracker(1)])

  0%|          | 0/299.0 [00:00<?, ?it/s]

Output()

ScalarField(grid=CartesianGrid(bounds=((0.0, 4000.0), (0.0, 4000.0), (0.0, 4000.0)), shape=(20, 20, 20), periodic=[False, False, False]), data=[[[0.27316114 0.27323714 0.27338122 ... 0.27668061 0.27650501 0.2764127 ]
  [0.27326113 0.27334127 0.27349541 ... 0.27679097 0.27660685 0.27650924]
  [0.27345501 0.27354327 0.27371677 ... 0.27700337 0.27679917 0.27669307]
  ...
  [0.27275604 0.27305209 0.27357347 ... 0.27582032 0.27563753 0.27554023]
  [0.2723323  0.27257369 0.27304697 ... 0.27555667 0.27539123 0.27530426]
  [0.27210907 0.27232479 0.27274294 ... 0.27541864 0.27526358 0.27518115]]

 [[0.27326521 0.27334495 0.27350162 ... 0.2767846  0.27660095 0.27650355]
  [0.2733671  0.27344947 0.27361368 ... 0.27690371 0.27670678 0.27660544]
  [0.27356505 0.27365437 0.27383487 ... 0.27712434 0.27691253 0.2767978 ]
  ...
  [0.27281489 0.27306981 0.27355626 ... 0.27592975 0.27573159 0.27563002]
  [0.27239146 0.27261337 0.27304159 ... 0.27564935 0.27547675 0.2753836 ]
  [0.27217121 0.27237452 0.27

https://py-pde.readthedocs.io/en/latest/manual/advanced_usage.html#expressions
